### Load dataset and rename columns
> Note: **Only AMST** are contained in df. HNK products where filtered.

In [ ]:
import pandas as pd
import numpy as np
from re import sub

def column_name_to_snake_case(s):
    s = sub(r"[\/( \- ) ]", '_', s)
    s = sub(r"(___)|(__)", '_', s).lower()
    return s if s[-1] != '_' else s[:-1]

df = pd.read_csv(r'..\data\raw\Heineken - Data Science Use Case.csv', parse_dates=['Date/Time'])
df = (df.drop(columns=df.columns[0])
        .rename(columns={col:column_name_to_snake_case(col) for col in df.columns[1:]})
        .query('product == "AMST"')
     )
print(len(df))

### Rows with null values

In [ ]:
nulls = df[pd.isnull(df).any(axis=1)]
print('len nulls:', len(nulls))


#### Analyzing target (color) null values
All color null values have no other null value besides the target column.
> Note: try to predict these values later

In [ ]:
# only null that are target 
target_nulls = nulls.query('color.isnull()')
print(target_nulls.drop('color', axis=1).isnull().sum(axis=0))

# saving to parse dtypes faster later 
dtypes = {col:target_nulls[col].dtype for col in target_nulls.columns}

target_nulls.to_csv('..\data\dropped\dropped_rows.csv', index=False)



#### Check non_target nulls
Null values from other column correspond to small percentage of total count of row count.

We will use interpolation with KNN to estimate missing values.

In [ ]:
non_target = nulls.query('color.notnull()')
print('non_target nulls len:', len(non_target), '\n')

non_target = (non_target.drop('color', axis=1)
                        .isnull().sum(axis=0)[lambda x: x >0]
                        .to_frame('null_count'))

non_target = (non_target.assign(len_df = len(df))
                        .assign(ratio= non_target.null_count/len(df)))

print(non_target)


# using interpolation to fill NAs on roast amount and ph
df = df.assign(**{col:df[col].interpolate(method='nearest') for col in ['roast_amount_kg', 'ph']})
print('\n\nnull values after interpolation')
pd.isnull(df).sum()[lambda x: x > 0]

### Selected data

In [ ]:
import seaborn as sns
selected = (df.select_dtypes(include='number')
             .dropna()
             .drop(columns=['job_id'])
             .dropna()
             )
feature_cols = selected.columns
selected.describe()


### Check normality of features and target
All features were found to be normally distributed besides roast color


In [ ]:
from scipy.stats import shapiro, kstest

print('len features:', len(selected))
shapiro_p_value = [shapiro(selected[col].values).pvalue for col in feature_cols]
kstest_p_value = [kstest(selected[col].values, 'norm').pvalue for col in feature_cols]

normality_tests = pd.DataFrame(
    {'columns': feature_cols, 'shapiro_p_value': shapiro_p_value, 'kstest_p_value': kstest_p_value})

normality_tests['failed'] = normality_tests.apply(lambda row: 'yes' if row['shapiro_p_value'] > 0.05 or row['kstest_p_value'] > 0.05 else 'no', axis=1)
normality_tests

> note: roast color is constant, so it will be dropped

In [ ]:
selected = selected.drop(columns='roast_color')
selected.to_csv('../data/selected/selected.csv', index=False)

#### Check normality of target
> note: use median for baseline model

In [ ]:
shapiro_p_value = [shapiro(selected['color'].values).pvalue]
kstest_p_value = [kstest(selected['color'].values, 'norm').pvalue]
target_normality = pd.DataFrame(
    {'columns': ['color'], 'shapiro_p_value': shapiro_p_value, 'kstest_p_value': kstest_p_value})
selected[['color']].hist()
for i in ['mean', 'median', 'mode']:
    print(f'{i}: {getattr(selected.color, i)()}')
target_normality

checking how null values are positioned in timeline

In [ ]:
df['base_amount_kg'] = df['1st_malt_amount_kg'] + df['2nd_malt_amount_kg']
# Multiplied by 2 just offset lines a little bit
df.roast_amount_kg = df.roast_amount_kg*2
df['color (target)'] = df.color

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,5)
ax = df.sort_values('date_time').reset_index().plot(x='date_time', y=['ph', 'roast_amount_kg', 'color (target)'])
ax.set(title='Position of null values in dataset')

# undo changes to dataset
df.roast_amount_kg = df.roast_amount_kg/2

### Target box plot


In [ ]:
ax = df[['color']].boxplot()
ax.set(title='Target box plot')

### Outlier Detection


In [ ]:
from scipy import stats


#apply the z-score method and get abs 
z_scores = np.abs(stats.zscore(selected))


threshold = 3
outliers = selected[z_scores > threshold]
print(f'{len(outliers[outliers.notnull().any(axis=1)])} rows contain at least one outlier')
outliers = outliers[outliers.notnull().any(axis=1)]
print('Outlier ratio:', f'{len(outliers)/len(selected):.2%}')
outliers

## Correlation plots

In [ ]:
from typing import Literal
import matplotlib.pyplot as plt
def plot_correlation_heatmap(df:pd.DataFrame, corr_method:Literal['pearson', 'spearman', 'kendall']):
    plt.rcParams["figure.figsize"] = (10,10)
    
    corr_df = df.dropna().corr(corr_method)
    ax = sns.heatmap(corr_df, 
                    vmin=-1, vmax=1, center=0, 
                    cmap='bwr', annot=corr_df.values,annot_kws={'fontsize':8})
    ax.set(title=f'{corr_method.title()} Correlation plot')
    
    # absolute correlation 
    plt.rcParams["figure.figsize"] = (20,5)
    corr_abs = (corr_df[['color']].abs()
                              .sort_values('color', ascending=False)
                              .reset_index(names=['columns'])
                              .query('columns != "color"'))
    ax2  = corr_abs.plot.bar(x='columns', y='color', )
    ax2.set(title=f'Absolute {corr_method.title()} Correlation by Feature')
    def addlabels(x,y):
        for i in range(len(x)):
            plt.text(i-0.2,y[i]+0.005,y[i])
    addlabels(corr_abs.index, [float('%.3f'%x) for x in corr_abs.color.values])

    return ax, ax2

plot_correlation_heatmap(df[feature_cols], 'pearson')

##### insights:
total_cold_wort is highly correlated with:

    - extract
    - woc_time

wk_time and temperature are highly correlated with each other

1st and 2nd malt amount are highly correlated with each other

In [ ]:
plot_correlation_heatmap(df[feature_cols], 'spearman')

##### Insights
total_cold_wort has many good correlations with:

    - woc_time
    - wk_time
    - whp_time
  
1st malt amount has good correlation with:

    - wk_temperature
    
2nd malt amount has good correlation with:

    - whp_rest_time

usar time_series_train_test_split

treinar com e sem outliers